In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI
import httpx
import os
import tenacity
import json
from tqdm.auto import tqdm
from pprint import pprint

BASE_DIR = Path.cwd().parent

In [10]:
load_dotenv(Path.cwd().parent / ".env")

True

In [58]:
SYSTEM_PROMPT = """
Вы - специализированная модель для генерации синтетических наборов данных, имитирующих логи пользовательских сессий в виртуальных машинах. Ваша задача - создавать разнообразные, реалистичные примеры как нормальной, так и вредоносной активности пользователей.
Формат генерируемых данных

Каждая запись должна содержать:

1. session_id: уникальный идентификатор сессии
2. timestamp: время выполнения команды в формате ISO 8601
3. user: имя пользователя
4. command: выполненная команда с аргументами
5. duration: длительность выполнения команды в секундах
6. exit_code: код завершения команды
7. cwd: текущая рабочая директория
8. label: метка "malicious" или "benign"


Паттерны вредоносной активности
Генерируйте примеры следующих типов вредоносного поведения:

1. Разведка системы:

- Сканирование файловой системы (find / -type f -perm -04000 -ls)
- Поиск конфигурационных файлов (locate *.conf, find / -name "*.config")
- Проверка сетевых подключений (netstat -antp, ss -tuln)
- Поиск учетных данных (grep -r "password" /var/)


2. Попытки повышения привилегий:

- Эксплуатация уязвимостей (./exploit, ./cve-2023-xxxx)
- Запуск с sudo (sudo -l, попытки sudo без пароля)
- Попытки изменения системных файлов (chmod u+s /bin/bash)


3. Скачивание вредоносного ПО:

- Использование wget/curl для загрузки файлов с подозрительных доменов
- Компиляция скачанного кода (gcc -o malware malware.c)


4. Маскировка активности:

- Удаление логов (rm -rf /var/log/*)
- Изменение временных меток файлов (touch -t)
- Использование обфусцированных команд (echo "cm0gLXJmIC92YXIvbG9nLyo=" | base64 -d | bash)


5. Установка бэкдоров:

- Создание скрытых пользователей
- Модификация crontab
- Установка SSH-ключей


Паттерны нормальной активности
Генерируйте примеры следующих типов нормального поведения:

1. Административные задачи:

- Установка легитимных пакетов (apt-get install nginx)
- Настройка сервисов (systemctl restart apache2)
- Управление пользователями (adduser,pprint usermod)


2. Разработка:

- Работа с системами контроля версий (git clone, git commit)
- Компиляция кода (make, cargo build)
- Запуск тестов (pytest, npm test)


3. Мониторинг и обслуживание:

- Проверка состояния системы (df -h, free -m, top)
- Просмотр логов (tail -f /var/log/syslog)
- Резервное копирование (rsync, tar czf)


4. Обычная работа с файлами:

- Навигация (cd, ls)
- Редактирование файлов (vim, nano)
- Копирование/перемещение (cp, mv)

Особые указания

1. Реалистичность сессий:

- Генерируйте логические последовательности команд
- Соблюдайте разумные интервалы между командами
- Учитывайте время суток (административные задачи чаще выполняются в рабочее время)


2. Разнообразие данных:

- Варьируйте имена пользователей и их поведение
- Создавайте смешанные сессии с преимущественно нормальным поведением и отдельными вредоносными действиями
- Имитируйте ошибки ввода и исправления


3. Распределение классов:
- Создавайте сложные случаи на границе классификации


4. Симуляция различных сред:

- Генерируйте логи для разных дистрибутивов Linux (Ubuntu, CentOS, Debian)
- Учитывайте различия в командах и путях к файлам


5. Метаданные:

- Добавляйте комментарии о причинах классификации конкретных сессий
- Указывайте типы имитируемой вредоносной активности

Формат вывода
Генерируйте данные в формате JSON-lines, где каждая строка представляет одну команду:

[
    {"session_id": "sid-1234", "timestamp": "2025-03-15T14:32:17Z", "command": "ls -la /etc", "duration": 0.12, "exit_code": 0, "cwd": "/home/admin", "label": "benign"},
    {"session_id": "sid-1121", "timestamp": "2025-02-11T15:32:17Z", "command": "cd ..", "duration": 0.01, "exit_code": 0, "cwd": "/home", "label": "benign"}
]

При необходимости группируйте последовательности команд в рамках одной сессии для удобства анализа.
Создавайте набор данных с высоким разнообразием для обеспечения устойчивости модели определения вредоносной активности к различным сценариям и тактикам атак.

Просто генерируй валидный json, не нужно писать каких-либо комментариев от себя. Мне просто нужен валидный json.
"""

In [39]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPEN_ROUTER_KEY"),
    http_client=httpx.Client(proxy="http://localhost:2080")
)

In [40]:
completion = client.chat.completions.create(
    model="openai/o3-mini-high",
    messages=[{"role": "system", "content": SYSTEM_PROMPT},{"role": "user", "content": "Сгенерируй пример"}],
    temperature=0.4,
)

In [37]:
pprint(json.loads(completion.choices[0].message.content), indent=2)

[ { 'command': 'cd /home/alice',
    'cwd': '/home',
    'duration': 0.05,
    'exit_code': 0,
    'label': 'benign',
    'session_id': 'sid-1001',
    'timestamp': '2025-04-01T09:15:00Z',
    'user': 'alice'},
  { 'command': 'ls -la',
    'cwd': '/home/alice',
    'duration': 0.1,
    'exit_code': 0,
    'label': 'benign',
    'session_id': 'sid-1001',
    'timestamp': '2025-04-01T09:15:05Z',
    'user': 'alice'},
  { 'command': 'vim notes.txt',
    'cwd': '/home/alice',
    'duration': 1.2,
    'exit_code': 0,
    'label': 'benign',
    'session_id': 'sid-1001',
    'timestamp': '2025-04-01T09:15:10Z',
    'user': 'alice'},
  { 'command': 'apt-get update',
    'cwd': '/root',
    'duration': 12.34,
    'exit_code': 0,
    'label': 'benign',
    'session_id': 'sid-1002',
    'timestamp': '2025-04-01T10:00:00Z',
    'user': 'admin'},
  { 'command': 'apt-get install nginx',
    'cwd': '/root',
    'duration': 45.67,
    'exit_code': 0,
    'label': 'benign',
    'session_id': 'sid-1002'

In [53]:
@tenacity.retry(
    wait=tenacity.wait_fixed(1),
    stop=tenacity.stop_after_attempt(5),
    reraise=True,
)
def generate() -> list[dict]:
    completion = client.chat.completions.create(
        model="openai/o3-mini-high",
        messages=[{"role": "system", "content": SYSTEM_PROMPT},{"role": "user", "content": "Сгенерируй пример"}],
        temperature=0.4,
    )
    return json.loads(completion.choices[0].message.content)

In [54]:
NUM_ITER = 3
DATA_FOLDER = BASE_DIR / "data"
generated_data = list()
for _ in tqdm(range(NUM_ITER)):
    generated_data = generate()
    generated_data.extend(generated_data)

  0%|          | 0/3 [00:00<?, ?it/s]

In [57]:
OUTPUT_FILE = DATA_FOLDER / "generated_data.json"
with open(OUTPUT_FILE, "w") as f:
    json.dump(generated_data, f, indent=2)

In [59]:
with open(OUTPUT_FILE, "r") as f:
    data = json.load(f)
data

[{'session_id': 'sid-normal-001',
  'timestamp': '2023-10-01T09:00:00Z',
  'user': 'admin',
  'command': 'apt-get update',
  'duration': 1.23,
  'exit_code': 0,
  'cwd': '/root',
  'label': 'benign'},
 {'session_id': 'sid-normal-001',
  'timestamp': '2023-10-01T09:01:15Z',
  'user': 'admin',
  'command': 'apt-get install nginx',
  'duration': 2.45,
  'exit_code': 0,
  'cwd': '/root',
  'label': 'benign'},
 {'session_id': 'sid-normal-001',
  'timestamp': '2023-10-01T09:03:00Z',
  'user': 'admin',
  'command': 'systemctl restart nginx',
  'duration': 0.5,
  'exit_code': 0,
  'cwd': '/root',
  'label': 'benign'},
 {'session_id': 'sid-mal-001',
  'timestamp': '2023-10-01T22:15:30Z',
  'user': 'bob',
  'command': 'find / -type f -perm -04000 -ls',
  'duration': 4.56,
  'exit_code': 0,
  'cwd': '/home/bob',
  'label': 'malicious'},
 {'session_id': 'sid-mal-001',
  'timestamp': '2023-10-01T22:17:45Z',
  'user': 'bob',
  'command': 'grep -r "password" /var/',
  'duration': 3.25,
  'exit_code':

In [60]:
data[0]

{'session_id': 'sid-normal-001',
 'timestamp': '2023-10-01T09:00:00Z',
 'user': 'admin',
 'command': 'apt-get update',
 'duration': 1.23,
 'exit_code': 0,
 'cwd': '/root',
 'label': 'benign'}